<a href="https://colab.research.google.com/github/jimmy-io/Insight_project/blob/master/Copy_of_MODAnet_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%cd /content/drive/My Drive/Insight/Project/fashionbeans

/content/drive/My Drive/Insight/Project/fashionbeans


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!pip install keras-maskrcnn

     |████████████████████████████████| 61kB 4.2MB/s 
  Created wheel for keras-maskrcnn: filename=keras_maskrcnn-0.2.2-cp36-none-any.whl size=41692 sha256=8093f6c465e0239f557ad9d9eddb640ad179616cd227771e0b1a32d35080ac67
  Stored in directory: /root/.cache/pip/wheels/0b/03/0e/652a317641021f3450b92b69567e64cc54597d71e809426be8
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=156780 sha256=3c5a73c002754a82b34f02bac5aab562e14cd00875b8bba79b0c14e3722f9204
  Stored in directory: /root/.cache/pip/wheels/d9/a5/98/87f8ed08913b184bb625dde1c1277a4dfe969c77b377abde69
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20485 sha256=a25c7af98c8e5a7669ed93e3c56335af03363420908b06132e3f465a69df37d4
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-maskrcnn keras-retinanet keras-resnet


In [6]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras
import keras_maskrcnn
import keras_retinanet 

# import keras_retinanet
from keras_maskrcnn import models
from keras_maskrcnn.utils.visualization import draw_mask
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_annotations
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import shutil 
import numpy as np
import time
import json



# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [7]:
# adjust this to point to your downloaded/trained model
model_path = os.path.join('/content/drive/My Drive/Insight/Project/MAsk-RCNN', 'resnet50_modanet.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {1: 'bag', 2: 'belt', 3: 'boots', 4: 'footwear', 5: 'outer', 6: 'dress', 7: 'sunglasses', 8: 'pants', 9: 'top', 10: 'shorts', 11: 'skirt', 12: 'headwear', 13: 'scarf/tie'}








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
box_ind is deprecated, use box_indices instead







/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "filtered_detections" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "filtered_detections" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "mask_submodel" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "mask_submodel" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWar

In [0]:
def cloth_identifier(input_imagefile):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  outputs = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  boxes  = outputs[-4][0]
  scores = outputs[-3][0]
  labels = outputs[-2][0]
  masks  = outputs[-1][0]

  # correct for image scale
  boxes /= scale

  masks_dic={}
  boxes_dic={}
  counter=0

  # visualize detections
  for box, score, label, mask in zip(boxes, scores, labels, masks):
    if score < 0.5:
        break
    

    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    mask = mask[:, :, label]
    draw_mask(draw, b, mask, color=label_color(label))
    
    masks_dic[str(counter)]=mask
    boxes_dic[str(counter)]=box

    counter+=1

    #caption = "{} {:.3f}".format(labels_to_names[label], score)
    #draw_caption(draw, b, caption)
    
  newfileneame=input_imagefile.split(".")[0]  
  plt.ioff()
  plt.figure(figsize=(15, 15))
  plt.axis('off')
  plt.imshow(draw)
  plt.savefig(str('maskedimages/')+str(newfileneame)+'-masked'+'.jpg',bbox_inches='tight', pad_inches=0)
  plt.close('all')
  cwd = os.getcwd()

  os.rename(str(cwd)+'/'+str(newfileneame)+'.jpg', str(cwd)+'/processedimages/'+str(newfileneame)+'.jpg')
  #print(str(cwd)+'/'+'masked'+str(newfileneame)+'.jpg')
  #plt.show()
  

  

  return draw, masks_dic, boxes_dic



In [17]:
new_img, masks_dic, boxes_dic = cloth_identifier('0+1.jpg')

processing time:  0.647172212600708


In [0]:
### To run cloth_identifier over all images in a directory 

directory = '/content/drive/My Drive/Insight/Project/fashionbeans'
counter=0
onlyfiles = next(os.walk(directory))[2]
masked_images_dic={}
masked_items_dic={}

for filename in os.listdir(directory):
  try:
     if filename.endswith('jpg') or filename.endswith('png'):
       print(filename)

       new_img, masks_dic, boxes_dic = cloth_identifier(filename)



       #masked_images_dic[key]=(new_img)
       #masked_items_dic[key]=(items_dic)
 
       counter+=1

       

       print("File#",counter,"of",len(onlyfiles),"is processed")

  except:
    print('x')

0+16.jpg
processing time:  0.6586079597473145
File# 1 of 2069 is processed
0+2.jpg
processing time:  0.6519021987915039
File# 2 of 2069 is processed
0+17.jpg
processing time:  0.6481752395629883
File# 3 of 2069 is processed
2+30.jpg
processing time:  0.6415183544158936
File# 4 of 2069 is processed
0+3.jpg
processing time:  0.6406831741333008
File# 5 of 2069 is processed
3+1.jpg
processing time:  0.6416363716125488
File# 6 of 2069 is processed
0+18.jpg
processing time:  0.6438584327697754
File# 7 of 2069 is processed
0+4.jpg
processing time:  0.6529138088226318
File# 8 of 2069 is processed
3+2.jpg
processing time:  0.6439874172210693
File# 9 of 2069 is processed
3+3.jpg
processing time:  0.6404833793640137
File# 10 of 2069 is processed
0+19.jpg
processing time:  0.6447179317474365
File# 11 of 2069 is processed
0+5.jpg
processing time:  0.6394388675689697
File# 12 of 2069 is processed
1+7.jpg
processing time:  0.6486895084381104
File# 13 of 2069 is processed
1+22.jpg
processing time:  0.

In [0]:
### This is the function to extract clothing items separately from an image, given 
### the image and masks and boxes (as dictionaries) extracted using the function 'cloth_identifier'
### and returns a separate image file for each identfied clothing, based on the masks

def cloth_extract(input_imagefile, masks_dic, boxes_dic):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # visualize detections

  items_dic={}
  counter=0

  for box, mask in zip(boxes_dic.values(), masks_dic.values()):

    b = box.astype(int)

    # resize to fit the box
    mask = mask.astype(np.float32)
    mask = cv2.resize(mask, (b[2] - b[0], b[3] - b[1]))

    # binarize the mask
    mask = (mask > 0.5).astype(np.uint8)

    # draw the mask in the image
    mask_image = np.zeros((draw.shape[0], draw.shape[1]), np.uint8)
    mask_image[b[1]:b[3], b[0]:b[2]] = mask
    mask = mask_image

    mask = (np.stack([mask] * 3, axis = 2))*draw

    items_dic[str(counter)] = mask
    counter+=1

    newfileneame=input_imagefile.split(".")[0]
    plt.ioff()
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(mask)
    plt.savefig(str('maskeditems/')+str(newfileneame)+'masked'+str(counter)+'.jpg',bbox_inches='tight', pad_inches=0)
    #plt.show()
    plt.close('all')
    cwd = os.getcwd()
    os.rename(str(cwd)+'/'+'masked_item-'+str(counter)+str(newfileneame)+'.jpg', str(cwd)+'/maskeditems/'+'masked_item-'+str(counter)+str(newfileneame)+'.jpg')
  
  return items_dic




In [0]:
items_dic = cloth_extract('01.jpg', masks_dic, boxes_dic)

In [0]:
### To run cloth_identifier and cloth_extract over all images in a directory 

directory = '/content/drive/My Drive/Insight/Project/Reddit images'
counter=0
onlyfiles = next(os.walk(directory))[2]
masked_images_dic={}
masked_items_dic={}

for filename in os.listdir(directory):
  try:
     if filename.endswith('jpg') or filename.endswith('png'):
       print(filename)

       new_img, masks_dic, boxes_dic = cloth_identifier(filename)

       items_dic = cloth_extract(filename, masks_dic, boxes_dic)
       cwd = os.getcwd()
       os.rename(str(cwd)+'/'+str(filename), str(cwd)+'/processed images/'+str(filename))
  
       #key = str(filename)

       #masked_images_dic[key]=(new_img)
       #masked_items_dic[key]=(items_dic)
 
       counter+=1

       

       print("File#",counter,"of",len(onlyfiles),"is processed")

  except:
    print('x')


0.97-1660-[WIWT] extra pockets.jpg
processing time:  2.2100725173950195
x
0.98-910-23.jpg
processing time:  0.6312954425811768
x
0.94-865-30.jpg
processing time:  0.6432268619537354
x
0.97-1660-1.jpg
processing time:  0.5364997386932373
x
0.97-809-41.jpg
processing time:  0.618093729019165
x
0.92-804-43.jpg
processing time:  0.5552768707275391
x
0.95-756-49.jpg
processing time:  0.7093136310577393
x
0.98-745-54.jpg
processing time:  0.6375555992126465
x
0.97-663-76.jpg
processing time:  0.6803734302520752
x
0.91-663-77.jpg
processing time:  0.6811275482177734
x
0.96-651-85.jpg
processing time:  0.5490546226501465
x
0.93-639-89.jpg
processing time:  0.6451027393341064
x
0.85-475-156.jpg
processing time:  0.7445173263549805
x
0.94-450-178.jpg
processing time:  0.7651948928833008
x
0.94-418-203.jpg
processing time:  0.6547856330871582
x
0.9-405-211.jpg
processing time:  0.6962285041809082
x
0.91-383-223.jpg
processing time:  0.6982672214508057
x
0.95-333-267.jpg
processing time:  0.576763

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


processing time:  0.6334488391876221
x
0.33-0-2652.jpg
processing time:  0.6266849040985107
x
0.5-0-2657.jpg
processing time:  0.6459164619445801
x
0.45-0-2673.jpg
processing time:  0.6325697898864746
x
0.5-0-2738.jpg
processing time:  0.6355092525482178
x
0.41-0-2785.jpg
processing time:  0.6947283744812012
x
0.29-0-2787.jpg
processing time:  0.648306131362915
x
0.4-0-2836.jpg
processing time:  0.6790065765380859
x
0.38-0-2876.jpg
processing time:  0.5539267063140869
x
0.42-0-2927.jpg
processing time:  0.6702218055725098
x
0.17-0-2928.jpg
processing time:  0.6476585865020752
x
0.26-0-2984.jpg
processing time:  0.6814289093017578
x
0.29-0-2989.jpg
processing time:  0.6545577049255371
x
0.44-0-3074.jpg
processing time:  0.6748368740081787
x
0.25-0-3083.jpg
processing time:  0.6496522426605225
x
0.17-0-3095.jpg
processing time:  0.581047534942627
x
0.43-0-3107.jpg
processing time:  0.645754337310791
x
0.39-0-3117.jpg
processing time:  0.6470308303833008
x
0.24-0-3144.jpg
processing time:

In [0]:
### To run cloth_identifier and cloth_extract over all images in a directory 

directory = '/content/drive/My Drive/Insight/Project/Reddit images'
counter=0
onlyfiles = next(os.walk(directory))[2]
masked_images_dic={}
masked_items_dic={}

for filename in os.listdir(directory):
  if filename.endswith('jpg') or filename.endswith('png'):
    print(filename)

    new_img, masks_dic, boxes_dic = cloth_identifier(filename)

    items_dic = cloth_extract(filename, masks_dic, boxes_dic)
    cwd = os.getcwd()
    os.rename(str(cwd)+'/'+str(filename), str(cwd)+'/processed images/'+str(filename))

 
    counter+=1

       

    print("File#",counter,"of",len(onlyfiles),"is processed")



0.95-756-49.jpg
processing time:  0.7251987457275391
File# 1 of 1204 is processed
0.98-745-54.jpg
processing time:  0.6313109397888184


ValueError: ignored